In [1]:
import os
root = "../../Foundation_of_Advanced_Quantitative_Marketing_Python"
os.chdir(root)
import numpy as np
import pandas as pd
np.random.seed(123)

# Simulation
simulate aggregate market share for BLP model

In [2]:
# -----------------------------
# Minimal BLP example (toy)
# -----------------------------

# market structure
T = 10          # number of markets (e.g., cities/quarters)
J = 8           # number of alternatives
I = 50000       # number of individuals for simulating shares

markets = np.repeat(np.arange(T), J)
products = np.tile(np.arange(J), T)

# characteristics x1, price p and unobserved quality xi
x1 = np.random.normal(size=T*J)
true_price = -1.5           # price (positive, in utility it will be -alpha * p)
true_beta_x1 = 1.0         # mean coefficient for x1
xi = 0.3 * np.random.normal(size=T*J)   # unobserved quality
print("True parameters:")
print(f"beta_price: {true_price}, beta_x1: {true_beta_x1}")

# make prices positively correlated with xi to create "endogeneity"
# base cost + affected by xi
base_cost = 2.0 + 0.3*np.random.normal(size=T*J)
prices = np.exp(0.2 + 0.6*x1 + 0.5*xi) + base_cost

# individual heterogeneity: add a random coefficient to price and x1
sigma_price_true = 1.0
sigma_x1_true = 0.5

# simulate market shares by averaging over I individuals
# each individual has their own random coefficients drawn from N(0, sigma^2)
nu_price = np.random.normal(size=(I, T))    # I x T
nu_x1 = np.random.normal(size=(I, T))

shares = np.zeros(T*J)
for t in range(T):
    idx = slice(t*J, (t+1)*J)
    delta = true_beta_x1 * x1[idx] + true_price * prices[idx] + xi[idx]
    mu_i = (sigma_price_true * prices[idx][None, :] * nu_price[:, [t]]) + (sigma_x1_true * x1[idx][None, :] * nu_x1[:, [t]])
    util = delta[None, :] + mu_i
    expu = np.exp(util)
    denom = 1 + np.sum(expu, axis=1, keepdims=True)
    pij = expu / denom
    s = pij.mean(axis=0)        # average individual choice probability -> market share
    # ensure shares are strictly in (0,1)
    s = np.clip(s, 1e-6, 1 - 1e-6)
    shares[idx] = s

# construct product table
df = pd.DataFrame({
    'market_ids': markets,
    'product_ids': products,
    'shares': shares,
    'prices': prices,
    'x1': x1,
})

# a simple instrument
def blp_instruments_simple(df):
    Z1, Z2 = [], []
    for t in range(T):
        idx = slice(t*J, (t+1)*J-1)
        x1_t = df.loc[idx, 'x1'].values
        p_t = df.loc[idx, 'prices'].values
        Z1.append((x1_t.sum() - x1_t).reshape(-1, 1))
        Z2.append((p_t.sum() - p_t).reshape(-1, 1))
    return np.vstack(Z1), np.vstack(Z2)

Zx1, Zp = blp_instruments_simple(df)
df['iv_x1_othersum'] = Zx1
df['iv_p_othersum']  = Zp
df['base_cost'] = base_cost
df.head()

True parameters:
beta_price: -1.5, beta_x1: 1.0


,market_ids,product_ids,shares,prices,x1,iv_x1_othersum,iv_p_othersum,base_cost
0,0,0,0.018466,2.774765,-1.085631,-2.008726,23.332472,2.006095
1,0,1,0.041913,3.945674,0.997345,-4.091702,22.161563,1.941811
2,0,2,0.040794,3.897148,0.282978,-3.377335,22.210089,2.040208
3,0,3,0.012271,2.769742,-1.506295,-1.588062,23.337495,2.211342
4,0,4,0.014747,3.023046,-0.578600,-2.515757,23.084191,2.199696


# Apply BLP model

In [49]:
import statsmodels.api as sm
from src import logit_boost as lgt
shares = df['shares'].values
outside = 1 - df.groupby('market_ids')['shares'].transform('sum').values
col_instruments = ['iv_x1_othersum', 'iv_p_othersum']
col_endo = ['prices']
col_exo = ['x1']
col_x = col_endo + col_exo
X = df[col_x].values
Z = df[col_instruments+col_exo].values
D = 1000
t = 10
p = 8
# X = sm.add_constant(X)
Z = sm.add_constant(Z)
blp_model = lgt.BLP(X, Z, shares, outside, D, t, p)
gamma_hat, beta_hat = blp_model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.30179D+00    |proj g|=  4.43317D+00



 This problem is unconstrained.


At iterate    1    f=  5.45438D-01    |proj g|=  1.43146D+00

At iterate    2    f=  3.83508D-01    |proj g|=  3.98669D-01

At iterate    3    f=  3.67307D-01    |proj g|=  2.47872D-01

At iterate    4    f=  2.38861D-01    |proj g|=  1.46565D+00

At iterate    5    f=  9.11091D-02    |proj g|=  9.12374D-01

At iterate    6    f=  4.55220D-02    |proj g|=  2.58514D-01

At iterate    7    f=  8.55657D-03    |proj g|=  2.73815D-01

At iterate    8    f=  6.07004D-04    |proj g|=  7.46359D-02

At iterate    9    f=  5.64055D-07    |proj g|=  1.28605D-03

At iterate   10    f=  7.39356D-09    |proj g|=  1.37188D-04

At iterate   11    f=  1.93471D-14    |proj g|=  4.02661D-07

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F    

In [50]:
blp_model.summary()

Estimated gamma: [1.0245751  0.19090908 1.28336647]
Estimated beta: [-1.6684138   1.26439252]
Final loss: 1.9347071107684504e-14


## "cheating" with best instrument (cost)
use 'base_cost' with noise as instrument

In [70]:
import statsmodels.api as sm
from src import logit_boost as lgt
shares = df['shares'].values
outside = 1 - df.groupby('market_ids')['shares'].transform('sum').values
col_instruments = ['base_cost','iv_x1_othersum', 'iv_p_othersum']  ## cheating to test the function
col_endo = ['prices']
col_exo = ['x1']
col_x = col_endo + col_exo
X = df[col_x].values
Z = df[col_instruments+col_exo].values
D = 1000
t = 10
p = 8
# add noise to the first column of Z
Z[:,0] = Z[:,0] + 0.01 * np.random.normal(size=Z.shape[0])
# Z = sm.add_constant(Z)
blp_model = lgt.BLP(X, Z, shares, outside, D, t, p)
gamma_hat, beta_hat = blp_model.fit()

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.98003D+00    |proj g|=  5.43224D+00

At iterate    1    f=  7.10986D-01    |proj g|=  2.99889D+00

At iterate    2    f=  3.02476D-01    |proj g|=  1.60344D+00

At iterate    3    f=  1.86979D-01    |proj g|=  5.97924D-01

At iterate    4    f=  1.23554D-01    |proj g|=  6.09050D-01

At iterate    5    f=  1.91091D-02    |proj g|=  5.85436D-01

At iterate    6    f=  3.48026D-03    |proj g|=  2.76829D-01

At iterate    7    f=  1.90074D-04    |proj g|=  4.73471D-02

At iterate    8    f=  2.34358D-07    |proj g|=  9.83042D-04

At iterate    9    f=  4.79134D-10    |proj g|=  6.42811D-05

At iterate   10    f=  1.39843D-13    |proj g|=  1.72449D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cau

In [71]:
blp_model.summary()

Estimated gamma: [ 0.93651949 -0.14850349  0.46022517]
Estimated beta: [-1.47236745  1.10222482]
Final loss: 1.3984271663408703e-13
